In [15]:
import pandas as pd
from datetime import datetime

In [ ]:
data_dir='/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_financial/ACD_financial_reports/financial_data/contracting_hours/'  #GAK_timespent_to_3_25_25.xlsx

df=pd.read_excel(io=(data_dir+'GAK_timespent_to_3_25_25.xlsx'))
df.to_csv('GAK_timespent_3_25_25to')

# Caputure time 'User', 'Time spent', 'Task name', 'Group' = Date, board'

df_sub=df[['Task name', 'Group', 'User', 'Time spent', 'Board']].copy()

# Cast group as a datetime and rename it 'Date'
df_sub['Date']=pd.to_datetime(df_sub['Group'], format='%Y-%m-%d')
# Make new columns expandng 'task name' to indivudual columns with cumulative sums

df_sub.drop(columns=['Group'], inplace=True)

In [ ]:
# Pivot the DataFrame so that task names become column headers and time spent are the values
df_pivoted = df_sub.pivot_table(index='Date', columns=['Task name', 'Board'], values='Time spent', aggfunc='sum')
# Display the first few rows of the melted DataFrame

# Get all the levels of the 'Board' level in the MultiIndex
board_levels = df_pivoted.columns.get_level_values('Board').unique()
print(board_levels)


Index(['ACD_business', 'ACD_Financial', 'ACD_education', 'UCBerkeley_work',
       'BDSG_operations', 'ACD_Projects'],
      dtype='object', name='Board')


Task name,ACD internal meetings,Acd finance tracking codebase,Axcel course revamp - curriculum generation,Axcel course revamp - meetings and admin,Axcel course revamp - outline work,Billing,CDC - NCIPC session 6- Graph RAG and adaptors demo,Mark Labowitz review work,UC Berkeley curriculum development,UC Berkeley emails meetings and organization,...,meetings,payroll,phising in detailed,privacy and Data Science security advocate,scoping projects,social media posting and commenting,staffing emails and scheduling etc,taxes and finance,vignette generation,you tube operations (edit and post)
Board,ACD_business,ACD_Financial,ACD_education,ACD_education,ACD_education,ACD_Financial,ACD_education,UCBerkeley_work,UCBerkeley_work,UCBerkeley_work,...,BDSG_operations,ACD_Financial,ACD_education,ACD_education,BDSG_operations,ACD_education,ACD_business,ACD_Financial,BDSG_operations,ACD_Projects
Date,,,,,,,,,,,,,,,,,,,,,
2025-01-02,NaN,2.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.35
2025-01-03,NaN,0.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.65
2025-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-07,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-08,3.98,1.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## filters and concatenation##

## boards =['ACD_business','ACD_Financial','ACD_education','ACD_Projects']
## date_range = [1/1/25, 2/30/25]

df_ACD = df_pivoted.xs(('ACD_business'), level='Board', axis=1)
df_ACD2 = df_pivoted.xs(('ACD_Financial'), level='Board', axis=1)
df_ACD3 = df_pivoted.xs(('ACD_education'), level='Board', axis=1)
df_ACD4 = df_pivoted.xs(('ACD_Projects'), level='Board', axis=1)
df_ACD_jn = df_ACD.join(df_ACD2).join(df_ACD3).join(df_ACD4)


#df_ACD_jn
df_ACD_jn=df_ACD_jn.loc['2025-01-02':'2025-02-28',]

print(df_ACD_jn.shape)

display(df_ACD_jn.columns)
#df_ACD_jn


(35, 19)


Index(['ACD internal meetings',
       'answering emails scheduling and slack etc - ACD',
       'contract generation and maintenance', 'managing work and tasks',
       'staffing emails and scheduling etc', 'Acd finance tracking codebase',
       'Billing', 'payroll', 'taxes and finance',
       'Axcel course revamp - curriculum generation',
       'Axcel course revamp - meetings and admin',
       'Axcel course revamp - outline work',
       'CDC - NCIPC session 6- Graph RAG and adaptors demo',
       'Web page updates', 'content editing', 'phising in detailed',
       'privacy and Data Science security advocate',
       'social media posting and commenting',
       'you tube operations (edit and post)'],
      dtype='object', name='Task name')

nan

In [ ]:
#def sum_trends(df_col):
for i in df_ACD_jn.columns:

    df_ACD_jn[i].fillna(0, inplace = True)
    df_ACD_jn[i]= df_ACD_jn[i].cumsum()
    

In [ ]:
work_tally=input('what should this work tally be called? (worker_dates_billabe/all)')

# create excel and append to it
excel_path = f'{work_tally}.xlsx'
with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    # Write the DataFrame to the Excel file
    #df_ACD_jn.to_excel(writer, sheet_name='Work Tally')

    # Access the workbook and worksheet objects
    workbook = writer.book
    worksheet = workbook.add_worksheet('Work Tally')

    # Start from the first cell below the header
    start_row = 0

    # Write headers for the new columns
    worksheet.write(start_row, 0, 'Description')
    worksheet.write(start_row, 1, 'Total Hours')
    start_row += 1

    # Append lines to the Excel file
    for i in df_ACD_jn.columns:
        worksheet.write(start_row, 0, i)  # Write the task name to the 'Description' column
        worksheet.write(start_row, 1, max(df_ACD_jn[i]))  # Write the max time spent to the 'Total Hours' column
        start_row += 1

print(f'Excel file {excel_path} created successfully.')

df_ACD_jn.to_csv('ACD_hrs.csv')

Excel file gkleemann_Jan25:feb25_allACD.xlsx created successfully.
